In [1]:
import pandas as pd
import gzip
from structuredata import list_inbound_outbound, import_data
from json_reader import read_json, write_gjson_into_file
from BuildDB.build_db import *
from bson import ObjectId
%matplotlib inline
from matplotlib import pyplot as plt
import math
from datetime import datetime, timedelta, time

In [2]:
directory = "./SampleData"

inb, outb = list_inbound_outbound(directory)

In [3]:
df = import_data(inb, outb)

Creating list of data frames...



In [17]:
print df.conversation.iloc[1][0]

  content     from_addr from_addr_type group  \
0    None  +27729426700           None  None   

                                     helper_metadata in_reply_to  \
0  {u'go': {u'conversation_type': u'jsbox', u'is_...        None   

                         message_id  message_type message_version provider  \
0  4ad06d5fef7d4365b12754c4ffe92b57  user_message        20110921     None   

                                    routing_metadata session_event  \
0  {u'go_hops': [[[u'TRANSPORT_TAG:truteq_4729:*1...           new   

                    timestamp       to_addr to_addr_type transport_metadata  \
0  2014-05-15 20:59:59.933044  *120*4729*1#         None                 {}   

          transport_name transport_type  
0  truteq_4729_transport           ussd  


In [4]:
conn, db, conversations_collection = mongo_database_setup()

Connected successfully to MongoDB, instance created!


In [5]:
object_ids = mongod_insert_df(df, conversations_collection)

KeyboardInterrupt: 

In [ ]:
channel1 = ['*120*7692*2#']
channel2 = ['*120*7692*3#']
channel3 = ['*120*4729#']

graph1 = ['2014-04-08', '2014-04-19', '2014-04-24', '2014-04-28']
graph2 = ['2014-04-09', '2014-04-20', '2014-04-25', '2014-04-29']
graph3 = ['2014-04-10', '2014-04-21', '2014-04-26', '2014-04-30']


channels = [channel1,channel2,channel3]
graphs = [graph1,graph2,graph3]

pcm_0 = {'2014-04-08':{}, '2014-04-19':{}, '2014-04-24':{}, '2014-04-28':{}}
pcm_1 = {'2014-04-09':{}, '2014-04-20':{}, '2014-04-25':{}, '2014-04-29':{}}
pcm_2 = {'2014-04-10':{}, '2014-04-21':{}, '2014-04-26':{}, '2014-04-30':{}}

pcms = [pcm_0,pcm_1,pcm_2]

min_date = '2014-04-08'
max_date = '0000-00-00 00:00:00.000000'


## determine pcm bursts 
for id_ in object_ids:

    doc = conversations_collection.find_one({"_id": ObjectId(id_)})

    user = doc['user']
    conversation = doc['conversation']

    num = len(conversation) 
    msg = ''.join(('msg',str(num)))
    t = conversation[msg]['timestamp'] 
    if t > max_date: max_date = t


    ## construct values for figure 1 on response frequency:

    if len(conversation)>=2 and conversation['msg1']['transport_type'] == 'ussd':

        channel = conversation['msg1']['from_addr']

        if channel in channels:

            num = channels.index(channel)
            graph = graphs[num]

            timestamp = conversation['msg2']['timestamp']

            dname = ''.join(('pcm_',num))
            if graph[0] <= timestamp < graph[1]:

                if timestamp in dname[graph[0]]:
                    dname[graph[0]][timestamp]  = dname[graph[0]][timestamp] + 1

                else:
                    dname[graph[0]][timestamp]  = 1

            elif graph[1] <= timestamp < graph[2]:
                if timestamp in dname[graph[1]]:
                    dname[graph[1]][timestamp]  = dname[graph[1]][timestamp] + 1

                else:
                    dname[graph[1]][timestamp]  = 1

            elif graph[2] <= timestamp < graph[3]:
                if timestamp in dname[graph[2]]:
                    dname[graph[2]][timestamp]  = dname[graph[2]][timestamp] + 1

                else:
                    dname[graph[2]][timestamp]  = 1

            elif graph[3] <= timestamp:
                if timestamp in dname[graph[3]]:
                    dname[graph[3]][timestamp]  = dname[graph[3]][timestamp] + 1

                else:
                    dname[graph[3]][timestamp]  = 1

In [ ]:
conn.close()

In [ ]:
start_date = datetime.strptime(min_date,"%Y-%m-%d").date()
end_date = datetime.strptime(max_date.split(' ')[0],"%Y-%m-%d").date()
range_dates = []

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

for single_date in daterange(start_date, end_date):
    range_dates.append(single_date.strftime("%Y-%m-%d"))


#graph1 (channel1)

fig, ax1 = plt.subplots()
x_axis = range_dates 

#build up y axis:
l = len(range_dates)
y_axis = [0]*l
y_axis1 = y_axis
y_axis1[0] = math.log(13333333)
y_axis1[11] = math.log(1000000)
y_axis1[16] = math.log(1000000)
y_axis1[20] = math.log(1000000)

ax1.plot_date(x_axis, y_axis1,'b-')

ax1.set_xlabel('date')
ax1.set_ylabel('pcm release over channel1 (log)', color='b')
ax1.tick_params('y',colors='b')

ax2 = ax1.twinx()

y_axis2 = y_axis

for pcm in pcm_0.values():
	for d,n in pcm.iteritems():
		day = d[-2:]
		y_axis2[day-8-1] = n


ax2.plot_date(x_axis,y_axis2,'r-')
ax2.set_ylabel('first response',color='r')
ax2.tick_params('y',colors='r')

fig.tight_layout()
plt.show()


#graph2 (channel2):

fig, ax1 = plt.subplots()
x_axis = range_dates 

#build up y axis:
l = len(range_dates)
y_axis = [0]*l
y_axis1 = y_axis
y_axis1[1] = math.log(13333333)
y_axis1[12] = math.log(1000000)
y_axis1[17] = math.log(1000000)
y_axis1[21] = math.log(1000000)

ax1.plot_date(x_axis, y_axis1,'b-')

ax1.set_xlabel('date')
ax1.set_ylabel('pcm release date over channel 2(log)', color='b')
ax1.tick_params('y',colors='b')

ax2 = ax1.twinx()

y_axis2 = y_axis

for pcm in pcm_0.values():
	for d,n in pcm.iteritems():
		day = d[-2:]
		y_axis2[day-8-1] = n


ax2.plot_date(x_axis,y_axis2,'r-')
ax2.set_ylabel('first response',color='r')
ax2.tick_params('y',colors='r')

fig.tight_layout()
plt.show()

#graph3 (channel3):
fig, ax1 = plt.subplots()
x_axis = range_dates 

#build up y axis:
l = len(range_dates)
y_axis = [0]*l
y_axis1 = y_axis
y_axis1[2] = math.log(13333333)
y_axis1[13] = math.log(1000000)
y_axis1[18] = math.log(1000000)
y_axis1[22] = math.log(1000000)

ax1.plot_date(x_axis, y_axis1,'b-')

ax1.set_xlabel('date')
ax1.set_ylabel('pcm release date over channel 3 (log)', color='b')
ax1.tick_params('y',colors='b')

ax2 = ax1.twinx()

y_axis2 = y_axis

for pcm in pcm_0.values():
	for d,n in pcm.iteritems():
		day = d[-2:]
		y_axis2[day-8-1] = n


ax2.plot_date(x_axis,y_axis2,'r-')
ax2.set_ylabel('first response',color='r')
ax2.tick_params('y',colors='r'e)

fig.tight_layout()
plt.show()


In [ ]:
#graph2 (channel2):

fig, ax1 = plt.subplots()
x_axis = range_dates 

#build up y axis:
l = len(range_dates)
y_axis = [0]*l
y_axis1 = y_axis
y_axis1[1] = math.log(13333333)
y_axis1[12] = math.log(1000000)
y_axis1[17] = math.log(1000000)
y_axis1[21] = math.log(1000000)

ax1.plot_date(x_axis, y_axis1,'b-')

ax1.set_xlabel('date')
ax1.set_ylabel('pcm release date over channel 2(log)', color='b')
ax1.tick_params('y',colors='b')

ax2 = ax1.twinx()

y_axis2 = y_axis

for pcm in pcm_0.values():
	for d,n in pcm.iteritems():
		day = d[-2:]
		y_axis2[day-8-1] = n


ax2.plot_date(x_axis,y_axis2,'r-')
ax2.set_ylabel('first response',color='r')
ax2.tick_params('y',colors='r')

fig.tight_layout()
plt.show()

In [ ]:
#graph3 (channel3):
fig, ax1 = plt.subplots()
x_axis = range_dates 

#build up y axis:
l = len(range_dates)
y_axis = [0]*l
y_axis1 = y_axis
y_axis1[2] = math.log(13333333)
y_axis1[13] = math.log(1000000)
y_axis1[18] = math.log(1000000)
y_axis1[22] = math.log(1000000)

ax1.plot_date(x_axis, y_axis1,'b-')

ax1.set_xlabel('date')
ax1.set_ylabel('pcm release date over channel 3 (log)', color='b')
ax1.tick_params('y',colors='b')

ax2 = ax1.twinx()

y_axis2 = y_axis

for pcm in pcm_0.values():
	for d,n in pcm.iteritems():
		day = d[-2:]
		y_axis2[day-8-1] = n


ax2.plot_date(x_axis,y_axis2,'r-')
ax2.set_ylabel('first response',color='r')
ax2.tick_params('y',colors='r')

fig.tight_layout()
plt.show()